# THIS IS A JUPYTER NOTEBOOK VERSION OF RT2 ASSIGNMENT

Hi, here is a Jupyter Notebook implementation. 

To start, the necessary libraries must be imported:

In [1]:
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal, MoveBaseActionFeedback
from actionlib_msgs.msg import GoalID, GoalStatusArray
from nav_msgs.msg import Odometry
from tf.transformations import *

import jupyros as jr
from jupyros import ros3d
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import math
import time
import os

import getch
import sys, tty
sys.path.append("../src/")
import option1
import option2
import option3

Now, the needed publishers and subscribers for the project must be re-written for the Jupyter version

In [5]:
# v = ros3d.Viewer()
# ros_connect = ros3d.ROSConnection(url="ws://localhost:9090") 
# tf_client = ros3d.TFClient(ros=ros_connect, fixed_frame='map')
# laser_view = ros3d.LaserScan(topic="/scan", ros=ros_connect, tf_client=tf_client) 
# map_view = ros3d.OccupancyGrid(topic="/map", ros=ros_connect, tf_client=tf_client, continuous=True) 
# path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=ros_connect, tf_client=tf_client) 
# urdf = ros3d.URDFModel(ros=ros_connect, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
# g = ros3d.GridModel()
# v.objects = [g, laser_view, map_view, path, urdf]
# v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

In [6]:
# ########### MAIN ############

# # User interface to give the user a set of options
# def welcome():
#     """
#     This is the instruction list for user
#     """
    
#     print("Whats Up? This is RT2 Assignment, and here is a Jupyter Notebook version for it.")
#     print("Press [1] for autonomous drive of the robot")
#     print("Press [2] for manual drive of the robot")
#     print("Press [3] for manual drive of the robot with collision avoidance")
#     print("Press [4] for exitting from the system")


# def main():

#     """
#     This function used to choose the driving mode by user
#     #The user message is passed to the service ``master``
#     """

#     exit_system = False
#     rospy.init_node('jupyter_node')
#     while not rospy.is_shutdown() and not exit_system:
#         welcome()
#         while True:
#             try:
#                 button = int(input("Choose drive options: "))
#                 break
#             except:
#                 print("Your input is not possible to obtain")
#         if button == 1:
#             while not option1.option_one():
#                 pass
#         if button == 2:
#             while not option2.option_two():
#                 pass
#         if button == 3:
#             while not option3.option_three():
#                 pass
#         elif button == 4:
#             exit_system = True
#         else:
#             print("Wrong button")

# if __name__ == '__main__':
#     main()

Whats Up? This is RT2 Assignment, and here is a Jupyter Notebook version for it.
Press [1] for autonomous drive of the robot
Press [2] for manual drive of the robot
Press [3] for manual drive of the robot with collision avoidance
Press [4] for exitting from the system
Choose drive options: 1
Autonomous drive is chosen
Insert x coordinate: 2
Insert y coordinate: 3
The coordinates were given: x = 2.00 and y = 3.00


[ERROR] [1654269087.564113, 203.308000]: bad callback: <function goal_status at 0x7fd85825d820>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_6722/2390110658.py", line 13, in goal_status
    if len(colors) < goal_counter + 1:   # If a new goal was sent, it increases the length of arrays.
NameError: name 'goal_counter' is not defined

[ERROR] [1654269087.578925, 203.308000]: bad callback: <function goal_status at 0x7fd85825d820>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_6722/2390110658.py", line 13, in goal_status
    if len(colors) < goal_counter + 1:   # If a new goal was sent, it increases the length of arrays.
NameError: name 'goal_counter' is not defined

[ERROR] [1654269087.725747, 203.461000]: bad callback: <function goal_status at 0x7f

IndexError: list index out of range

In [ ]:
display(sub_odom)

Buttons to control the robot

In [10]:
vel = Twist()
third = False
ok_left = True
ok_straight = True
ok_right = True

vl = 1
publ = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)

straight_button = widgets.Button(description = 'Up',
                                 style=dict(font_style='italic',
                                            font_weight='bold',
                                            font_variant="small-caps", 
                                            text_color='red', 
                                            text_decoration='underline')
                                )

back_button = widgets.Button(description = 'Down',
                                 style=dict(font_style='italic',
                                            font_weight='bold',
                                            font_variant="small-caps", 
                                            text_color='red', 
                                            text_decoration='underline')
                                )

left_button = widgets.Button(description = 'Left',
                                 style=dict(font_style='italic',
                                            font_weight='bold',
                                            font_variant="small-caps", 
                                            text_color='red', 
                                            text_decoration='underline')
                                )


right_button = widgets.Button(description = 'Right',
                                 style=dict(font_style='italic',
                                            font_weight='bold',
                                            font_variant="small-caps", 
                                            text_color='red', 
                                            text_decoration='underline')
                                )

increase_button = widgets.Button(description = 'Increase Velocities', 
                                 layout=Layout(width='60%', height='50px'), 
                                 button_style='info')
increase_button.style.button_color = 'green'

reset_button = widgets.Button(description = 'Reset Velocities', 
                              layout=Layout(width='60%', height='50px'), 
                              button_style='warning')

stop_button = widgets.Button(description = 'Stop', 
                             layout=Layout(width='60%', height='50px'), 
                             button_style='danger') 

def straight_button_clicked(b):
    global vel
    global vl
    global publ
    
    if ok_straight == True:
        vel.linear.x = vl
        vel.linear.y = 0
        vel.linear.z = 0
        vel.angular.x = 0
        vel.angular.y = 0
        vel.angular.z = 0
        publ.publish(vel)
    else:
        print("Command 'Go Straight' disabled")
    
straight_button.on_click(straight_button_clicked)

def back_button_clicked(b):
    global vel
    global vl
    global publ
    
    vel.linear.x = -vl
    vel.linear.y = 0
    vel.linear.z = 0
    vel.angular.x = 0
    vel.angular.y = 0
    vel.angular.z = 0
    publ.publish(vel)
    
back_button.on_click(back_button_clicked)  

def left_button_clicked(b):
    global vel
    global vl
    global publ
    
    if ok_left == True:
        vel.linear.x = 0
        vel.linear.y = 0
        vel.linear.z = 0
        vel.angular.x = 0
        vel.angular.y = 0
        vel.angular.z = vl
        publ.publish(vel)
    else:
        print("Command 'Go Left' disabled")

left_button.on_click(left_button_clicked)
    
def right_button_clicked(b):
    global vel
    global vl
    global publ

    
    if ok_right == True:
        vel.linear.x = 0
        vel.linear.y = 0
        vel.linear.z = 0
        vel.angular.x = 0
        vel.angular.y = 0
        vel.angular.z = -vl
        publ.publish(vel)
    else:
        print("You can't go right.")

right_button.on_click(right_button_clicked)
    
def increase_v_button_clicked(b):
    global vl
    vl = vl + 1

increase_button.on_click(increase_v_button_clicked)

def reset_v_button_clicked(b):
    global vl
    vl = 1

reset_button.on_click(reset_v_button_clicked)

def stop_button_clicked(b):
    vel.linear.x = 0
    vel.linear.y = 0
    vel.linear.z = 0
    vel.angular.x = 0
    vel.angular.y = 0
    vel.angular.z = 0
    publ.publish(vel)
    
stop_button.on_click(stop_button_clicked)

row1=widgets.HBox([straight_button, left_button, back_button, right_button ])
row3=widgets.HBox([stop_button])
row4=widgets.HBox([increase_button])
row5=widgets.HBox([reset_button])

Modalities

In [11]:


rospy.init_node("jupyter_ui")
output = widgets.Output()

des_x = widgets.FloatText(
    value=0,
    description='X:',
    disabled=False
)
des_y = widgets.FloatText(
    value=0,
    description='Y:',
    disabled=False
)

boxes = widgets.HBox([des_x, des_y])

cancel_button = widgets.Button(description = 'Click Me to cancel the goal!', layout=Layout(width='50%', height='80px'), button_style='danger')

def cancel_button_clicked(b):
    rospy.set_param('active', 0)
    
    
cancel_button.on_click(cancel_button_clicked)
    
ok_button = widgets.Button(description = 'Send Goal', layout=Layout(width='30%', height='30px'), button_style='')

def ok_button_clicked(b):

    rospy.set_param('active', 0)
    rospy.set_param('des_pos_x', des_x.value)
    rospy.set_param('des_pos_y', des_y.value)
    rospy.set_param('active', 1) 
    
ok_button.on_click(ok_button_clicked)
    
button_first_mod = widgets.Button(description = 'First modality, put your goal coordinates!', layout=Layout(width='50%', height='80px'))
button_first_mod.style.button_color = 'yellow'

button_second_mod = widgets.Button(description = 'Second modality, move freely your robot!',
                                   layout=Layout(width='50%', height='80px'))
button_second_mod.style.button_color = 'orange'

button_third_mod = widgets.Button(description = 'Third modality, move freely your robot assisted!', layout=Layout(width='50%', height='80px'), button_color='salmon')
button_third_mod.style.button_color = 'red'


def first_button_clicked(b):
    back_to_menu_mod()
    print("You chose the first controlling modality")
    third = False
    display(des_x)
    display(des_y)
    display(ok_button)
    display(cancel_button)
    
button_first_mod.on_click(first_button_clicked)

def second_button_clicked(b):
    back_to_menu_mod()
    third = False
    rospy.set_param('active', 0)
    rospy.set_param('active', 2)
    display(row1)
    display(row3)
    display(row4)
    display(row5)
    
    
button_second_mod.on_click(second_button_clicked)

def clbk_laser(msg):
    global vel
    global ok_left
    global ok_right
    global ok_straight
    global publ

    right = min(min(msg.ranges[0:143]), 1)      # Detects obstacles on the right of the robot
    front = min(min(msg.ranges[288:431]), 1)    # Detects obstacles in front of the robot
    left = min(min(msg.ranges[576:719]), 1)     # Detects obstacles on the left of the robot

    if right != 1.0 and vel.angular.z < 0:
        ok_right =False
        vel.angular.z = 0

    else:         
        ok_right =True

    if front != 1.0 and vel.linear.x > 0:  
        ok_straight = False
        vel.linear.x = 0

    else:              
        ok_straight =True

    if left != 1.0 and vel.angular.z > 0:    
        ok_left =False
        vel.angular.z = 0

    else:            
        ok_left =True
        
    publ.publish(vel)

def third_button_clicked(b):
    back_to_menu_mod()
    third=True
    rospy.set_param('active', 0)
    rospy.set_param('active', 3)
    display(row1)
    display(row3)
    display(row4)
    display(row5)
    
button_third_mod.on_click(third_button_clicked)

def back_to_menu_mod():
    clear_output(wait=True)
    display(buttons)

sub = rospy.Subscriber('/scan', LaserScan, clbk_laser)
buttons = widgets.HBox([button_first_mod, button_second_mod, button_third_mod])
display(buttons)

ROSException: rospy.init_node() has already been called with different arguments: ('jupyter_node', ['/home/bzwayne/.local/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/home/bzwayne/.local/share/jupyter/runtime/kernel-033331d1-8930-4f03-bf8c-6314ccb66ef5.json'], False, None, False, False)

Plot graphs for odometry, laser, and goal status

In [ ]:
import rospy
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Layout
import actionlib
from geometry_msgs.msg import Twist
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from tf import transformations
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
from matplotlib.animation import FuncAnimation
from sensor_msgs.msg import LaserScan
from actionlib_msgs.msg import GoalStatusArray


angles = []

class Visualiser:
    def __init__(self):
        
        self.fig_laser = plt.figure(figsize=(6,6))
        self.ax_laser = plt.subplot(111, polar=True)
        self.ax_laser.set_thetalim(-np.pi/2,np.pi/2)
        self.ax_laser.set_theta_zero_location("N")
       
        self.ln_laser, = self.ax_laser.plot([],[],'bo')
        
        
        self.fig_odom, self.ax_odom = plt.subplots()
        plt.grid(True)
        self.ln_odom, = plt.plot([] ,[], 'bo')
        
        self.achieved = False
        self.goal_counter = 0
        self.non_reached_goal_counter = 0
        self.laser  = [] 
        self.x_pos, self.y_pos = [], []

        self.fig_target, self.ax_target = plt.subplots()
        plt.grid(True)
        self.status_list = [0, 0]
        self.data = {'Reached Goals': self.goal_counter, 
                     'Unreached Goals':self.non_reached_goal_counter, 
                     'Sent Goals': self.goal_counter + self.non_reached_goal_counter}
        
        self.target = list(self.data.keys())
        self.values = list(self.data.values())
        self.ln_target = plt.bar(self.target, self.values, color=("green", "red", "blue"))
        
              
    def plot_init(self):
        self.ax_laser.set_title("Robot Laser")
        self.ax_laser.set_ylabel("Distance to obstacles",fontweight="bold")
        
        self.ax_odom.set_xlim(-20, 20)
        self.ax_odom.set_ylim(-20, 20)

        self.ax_odom.set_title("Robot Position",fontweight="bold")
        self.ax_odom.set_ylabel("Y",fontweight="bold")
        self.ax_odom.set_xlabel("X",fontweight="bold")

        self.ax_target.set_ylim(0, 10)
        
        return self.ln_laser , self.ln_odom, self.ln_target
    
    def scan_callback(self, msg):
        global angles
        angles = list(np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment))
        self.laser=list(msg.ranges)
        
    def odom_callback(self, msg):
        self.y_pos.append(msg.pose.pose.position.y)
        self.x_pos.append(msg.pose.pose.position.x)
        
    def goal_cb(self, msg): 

        try: 

            self.status_list.append(msg.status_list[0].status)

            if self.status_list[-1] == 3 and self.status_list[-2] != 3:
                self.goal_counter = self.goal_counter + 1

            elif self.status_list[-1] == 2 and self.status_list[-2] != 2:
                self.non_reached_goal_counter = self.non_reached_goal_counter + 1

            elif self.status_list[-1] == 4 and self.status_list[-2] != 2:
                self.non_reached_goal_counter = self.non_reached_goal_counter + 1

        except IndexError:
            pass
            
            
    def update_plot_odom(self, frame):
        x = self.x_pos
        y = self.y_pos
        self.ln_odom.set_data(x, y)
        return self.ln_odom
    
    def update_plot_target(self, frame):
        x = self.goal_counter
        y = self.non_reached_goal_counter
        z = self.goal_counter + self.non_reached_goal_counter
        self.updated_data = {'Reached Goals': x, 
                             'Non-reached Goals': y, 
                             'Total Sent Goals': z}
        
        h = list(self.updated_data.values())
        q = list(self.updated_data.keys())
        
        self.ln_target = plt.bar(q, h, color=("green", "red", "blue"))
        
        return self.ln_target
    
    def update_polar(self, frame):
        global angles    
        self.ax_laser.set_rmax(20)
        self.ln_laser.set_data(angles,self.laser)
        return self.ln_laser
    

vis = Visualiser()
sub_scan = rospy.Subscriber("/scan", LaserScan, vis.scan_callback)
sub_odom = rospy.Subscriber("/odom", Odometry, vis.odom_callback)
sub_status = rospy.Subscriber("/move_base/status", GoalStatusArray, vis.goal_cb)

ani_laser = FuncAnimation(vis.fig_laser, vis.update_polar, blit = True)
ani_odom = FuncAnimation(vis.fig_odom, vis.update_plot_odom, init_func=vis.plot_init)
ani_target = FuncAnimation(vis.fig_target, vis.update_plot_target, init_func=vis.plot_init, interval=100)

plt.show(block = True)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
